In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [ ]:
def crawl_call():
    result_list = []
    for num in range(1, 530):
        url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
        params = {
            'page' : num
        }
        resp = requests.get(url, params = params)
        soup = BeautifulSoup(resp.content)
        tr_list = soup.find_all('tr')[1:]
        for tr in tr_list:
            time = tr.find_all('td')[0].text.strip()
            a = time.split('.')
            time2 = date(int(a[0]), int(a[1]), int(a[2]))
            price = tr.find_all('td')[1].text

            new_dict = {
                'date' : time2,
                'price' : price
            }
            result_list.append(new_dict)

    df = pd.DataFrame(result_list)
    df = df.set_index('date')
    
    return df

# 데이터 전처리
 - 공휴일 콜금리 데이터 처리
 - 한달 전 데이터와 비교하여 상승 / 0 / 하락 라벨링

### 비어있는 콜금리 데이터 처리

In [ ]:
import pandas as pd
from datetime import date, timedelta

In [ ]:
def fill_call(df):
    date_data = pd.DataFrame(columns = ['date'])
    start = date(2004, 8, 20)
    end = date(2019, 7, 19)
    for n in range(int((end - start).days) + 1):
        date_data.loc[n, 'date'] = start + timedelta(n)
        
    call = date_data.merge(df, on = 'date', how = 'outer')
    call['price'] = call['price'].fillna(method='ffill')
    call = call.set_index('date')
    
    return call

### 라벨링

In [ ]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)

            if df.loc[time].price > df.loc[time2.date()].price:
                df_ud['up_down'][i] = '상승'
            elif df.loc[time].price == df.loc[time2.date()].price:
                df_ud['up_down'][i] = 0
            else:
                df_ud['up_down'][i] = '하락'
    
    call = df.join(df_ud, on = 'date', how = 'outer')
    call = call.loc[date(2005, 1, 1):]
    return call